## Project 03 Report

This notebook is a step-by-step guide for setting up a data pipeline that streams events from a web server into a database.  The major parts of the data pipeline include Flask, Kafka, Spark, HDFS, Hive, and Presto.  For this project's scope, a fictitious game uses the Flask API webserver to write user-generated data into Kafka.  Kafka's topic "events" ingests the user-events.  Using Spark, a data sync is set up to stream data into HDFS every ten seconds.  The Hive metastore is used to create a table structure for the join guild and purchase sword events that are then populated with data from HDFS using Presto.  The last section of the report discusses business insights derived from the user data--available via Presto queries. 

#### 1.0 | Spin Up Cluster

##### 1.1. 
Please see the docker-compose.yml file for more information about the data pipeline setup.  
&nbsp;  
```cd w205/project-3-samueljgomez/```  
```docker-compose up -d```
___

#### 2.0 | Start the Flask API Webserver 
##### 2.1  
I am using the mids container for the Flask app.  The env variable designates the location of the Flask app.  I am invoking flask run using the host option to connect to the server from the outside if needed. The game_api.py file contains the routes for the game.  
&nbsp;  
```docker-compose exec mids env FLASK_APP=/w205/project-3-samueljgomez/game_api.py flask run --host 0.0.0.0``` 
___

#### 3.0 | Setup Kafka Topic for Streaming
##### 3.1
This command is used to set up the kafka topic for streaming.  The name of the topic is events.  I am not using the -e option to keep the topic running continuously.  Kafka will now watch for events.  
&nbsp;  
```docker-compose exec mids kafkacat -C -b kafka:29092 -t events -o beginning```  
___

#### 4.0 | Run Spark Jobs
##### 4.1  
These lines of code send the write_swords_stream.py and write_guild_stream.py jobs to spark--please see those files for more about the content. Instead of using spark-pyspark I am using spark-submit. The mount point for spark is in w205, so I use only w205 when searching the directory.  These functions takes some time to run.  It is a good practice to keep in mind the overhead associated with certain tools.  
&nbsp;  
```docker-compose exec spark spark-submit /w205/project-3-samueljgomez/write_swords_stream.py```

```docker-compose exec spark spark-submit /w205/project-3-samueljgomez/write_guild_stream.py```
___

#### 5.0 | Look Into Hadoop Files
##### 5.1. 
Using this command I can see the tables for sword purchases and guild joins were created in the hadoop file system.  
&nbsp;  
```docker-compose exec cloudera hadoop fs -ls /tmp/```
___

#### 6.0 | Create Tables in Hive
##### 6.1. 
The hive metastore is used to track schema and create tables.  Here I am entering the hive console to create the guild_joins and sword_purchases tables.  
&nbsp;  
```docker-compose exec cloudera hive```    
##### 6.2. 
Below I am create two tables and registering the schema for sword purchases and guild joins. Note guild string, sword_type sting, and sword_cost int are columns in the tables that I created for guild joins and sword purchases respectively.  The compression used is snappy and the location in hdfs is specified.  There is no data stored in hive, only the table description.  
##### 6.2.1
##### hive metastore guild_joins
```create external table if not exists default.guild_joins (raw_event string, timestamp string, Accept string, Host string, User_Agent string, event_type string, guild string) stored as parquet location '/tmp/guild_joins'  tblproperties ("parquet.compress"="SNAPPY");```

```OK
Time taken: 0.917 seconds```  
##### 6.2.2
##### hive metastore sword_purchases
```create external table if not exists default.sword_purchases (raw_event string, timestamp string, Accept string, Host string, User_Agent string, event_type string, sword_type string, sword_cost int) stored as parquet location '/tmp/sword_purchases'  tblproperties ("parquet.compress"="SNAPPY");```
```OK
Time taken: 0.077 seconds```
___

#### 7.0 | Generate Events Using Curl and Apache Bench
##### 7.1  
Using this command I can generate a user event.   
&nbsp;   
```jupyter@midsw205:~/w205/project-3-samueljgomez$ docker-compose exec mids curl http://localhost:5000/join_guild```    
&nbsp;  
The response output lets the user know what guild was joined.  
&nbsp;   
```You have joined Blueblades!```    
&nbsp;  
To continuously generate events, I created the events_stream.sh file--please take a look for more information.  Running this command generates fifty events of join guild and purhcase sword with unique user IDs.

```bash events/stream_events.sh```
___

#### 8.0 | Watch Event Stream in Flask and Kafka. 
##### 8.1 Flask stream.
Below is a flask terminal output when an event occurs. 
&nbsp;   
```127.0.0.1 - - [02/Aug/2020 06:56:20] "GET /join_guild HTTP/1.0" 200 -```
```127.0.0.1 - - [02/Aug/2020 06:56:21] "GET /join_guild HTTP/1.0" 200 -```
```127.0.0.1 - - [02/Aug/2020 06:56:22] "GET /join_guild HTTP/1.0" 200 -```
```127.0.0.1 - - [02/Aug/2020 06:56:22] "GET /join_guild HTTP/1.0" 200 -```

##### 8.2 Kafka stream using curl.
The command line output in Kafka looks like this when an event is generated using the curl command.  The host is the user/player.  The guild is the guild that the user joined.  I created this functionality using the join_guild and guild_assign functions in the game_api.py file.  The event_type is also within the join_guild function.  It attaches the event type to the specific generated event.  The event type is used for allocating events to tables in hdfs, querying, etc.  The accept field is included as well as the user-agent field.  In this case the user-agent is curl.  
&nbsp;  
```{"Host": "localhost:5000", "guild": "Blueblades", "event_type": "join_guild", "Accept": "*/*", "User-Agent": "curl/7.47.0"}```  
##### 8.3 Kafka stream using Apache Bench.
In this case the user-agent is Apache Bench.  
Notice that I added sword type and sword cost to the purhcase sword event.  
&nbsp; 
```{"Host": "user1.comcast.com", "guild": "Blueblades", "event_type": "join_guild", "Accept": "*/*", "User-Agent": "ApacheBench/2.3"}
{"Host": "user1.att.com", "guild": "Triad", "event_type": "join_guild", "Accept": "*/*", "User-Agent": "ApacheBench/2.3"}
{"Host": "user2.comcast.com", "guild": "Triad", "event_type": "join_guild", "Accept": "*/*", "User-Agent": "ApacheBench/2.3"}
{"Host": "user2.att.com", "guild": "Triad", "event_type": "join_guild", "Accept": "*/*", "User-Agent": "ApacheBench/2.3"}```

```{"event_type": "purchase_sword", "sword_cost": 10, "Accept": "*/*", "User-Agent": "ApacheBench/2.3", "Host": "user1.comcast.com", "sword_type": "crusader"}
{"event_type": "purchase_sword", "sword_cost": 5, "Accept": "*/*", "User-Agent": "ApacheBench/2.3", "Host": "user1.att.com", "sword_type": "templer"}
{"event_type": "purchase_sword", "sword_cost": 10, "Accept": "*/*", "User-Agent": "ApacheBench/2.3", "Host": "user2.comcast.com", "sword_type": "crusader"}
{"event_type": "purchase_sword", "sword_cost": 10, "Accept": "*/*", "User-Agent": "ApacheBench/2.3", "Host": "user2.att.com", 
"sword_type": "crusader"}```
___

#### 9.0 | Use Presto for Business Insights
##### 9.1
##### This command opens the presto console.  I will query HDFS using this tool and infer the schema from hive.
```docker-compose exec presto presto --server presto:8080 --catalog hive --schema default```
##### 9.2
##### Display tables
```show tables;```
```
      Table      
-----------------
 guild_joins     
 sword_purchases 
``` 
##### 9.3
##### Describe table guild_joins.
```describe guild_joins;```
```
   Column   |  Type   | Comment 
------------+---------+---------
 raw_event  | varchar |         
 timestamp  | varchar |         
 accept     | varchar |         
 host       | varchar |         
 user_agent | varchar |         
 event_type | varchar |         
 guild      | varchar |         
```  

##### 9.4
##### Describe table sword_purchases.
```describe sword_purchases;```
```
   Column   |  Type   | Comment 
------------+---------+---------
 raw_event  | varchar |         
 timestamp  | varchar |         
 accept     | varchar |         
 host       | varchar |         
 user_agent | varchar |         
 event_type | varchar |         
 sword_type | varchar |         
 sword_cost | integer |         
```  
##### 9.5
##### One hundred user have join a guild.
```select count(distinct(host)) num_hosts from guild_joins;```
```
 num_hosts 
-----------
       100 
```
##### 9.6
##### The most popular guild is Blueblades.
```select guild, count(guild) as num_members from guild_joins group by guild order by 2 desc;```
```
    guild     | num_members 
--------------+-------------
 Blueblades   |          25 
 Triad        |          23 
 Bouldergarde |          18 
 Madvale      |          18 
 Hellforge    |          16 
 ```
##### 9.7
##### One hundred swords have been purhcased throughout gameplay.
```select count(sword_type) num_swords from sword_purchases;```
```
 num_swords 
------------
        100 
```

##### 9.8
##### The templer sword brought in the most revenue.  It was also the most purchased sword.  The most expensive sword, light saber, was only purchased by one user.
```select sword_type, count(sword_type) num_swords, sum(sword_cost) sum_sword_cost from sword_purchases group by sword_type;```
```
  sword_type   | num_swords | sum_sword_cost 
---------------+------------+----------------
 templer       |         83 |            415 
 crusader      |         14 |            140 
 hattori hanzo |          2 |             50 
 light saber   |          1 |             50
```
___

#### 10.0 | Business Case
Our user data reveals that most of the users are joining the Bludblades guild.   It would be a good idea to make users fulfill certain requirements like experience, accomplishments, etc. to join this guild.  User preference to join this guild might create more user gameplay if there were requirements to join.  We can also see that one hundred different users have joined a guild.

From the query ran on swords, we learned that the most popular sword is the templer sword.  This sword also generated the most revenue.  More revenue could be generated from the hattori hazo and light saber swords if promotions were offered.  This may also motivate users to purchase more than one sword and play more/spend more in order to earn the tokens to purchase these sword types.


Using post methods I could create more interaction with the end users.  For example, if a user requested a sword, I could prompt the user with the options of swords available.  I could also include a price for the sword and execute a trasaction by removing coins from their account.  Unfortunatley, I was not able to accomplish this level of functionality.  If I had more time, I would have enjoyed setting this up.